<a href="https://colab.research.google.com/github/CesarChalcoElias/prompting-techniques/blob/main/04_Ambiguity_Clarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ambiguity and Clarity

### **Overview**  
This tutorial focuses on two critical aspects of prompt engineering: identifying and resolving ambiguous prompts, and techniques for writing clearer prompts. These skills are essential for effective communication with AI models and obtaining more accurate and relevant responses.

### **Motivation**  
Ambiguity in prompts can lead to inconsistent or irrelevant AI responses, while lack of clarity can result in misunderstandings and inaccurate outputs. By mastering these aspects of prompt engineering, you can significantly improve the quality and reliability of AI-generated content across various applications.

## Setup

In [ ]:
!pip install langchain-openai langchain-core -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os
import random
from collections import Counter
from google.colab import userdata

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
llm = ChatOpenAI(model_name="gpt-5-nano")

## Identifying Ambiguous Prompts

In [ ]:
ambiguous_prompts = [
    "Tell me about the bank.",
    "What's the best way to get to school?",
    "Can you explain the theory?"
]

for prompt in ambiguous_prompts:
    analysis_prompt = f"Analyze the following prompt for ambiguity: '{prompt}'. Explain why it's ambiguous and list three possible interpretations."
    print(f"Prompt: {prompt}")
    print(llm.invoke(analysis_prompt).content)
    print("-" * 50)

Prompt: Tell me about the bank.
Ambiguity explanation:
- The word bank is highly polysemous (has multiple common senses), and “the bank” without context could refer to any of them. The prompt also lacks domain, location, or whether a specific bank is meant. This makes it unclear what information is being requested and about which particular entity or feature.

Three possible interpretations:
1) Financial institution
- What this would mean: Information about a bank as a place to manage money, including services (accounts, loans, mortgages, ATMs), how banks operate, regulatory protections (like FDIC insurance in the U.S.), fees, and typical customer considerations.
- Why plausible: “Tell me about the bank” is a natural phrasing when people discuss banks or financial institutions.

2) Riverbank or land beside a body of water
- What this would mean: Descriptions of the land along a river or lake—its geology, soil, vegetation, erosion processes, flood risks, habitats for wildlife.
- Why pla

Clearly there's a lot of ambiguity in these user prompts, and the llm can actually notice that.

## Resolving Ambiguity

In [ ]:
def resolve_ambiguity(prompt, context):
    """
    Resolve ambiguity in a prompt by providing additional context.

    Args:
    prompt (str): The original ambiguous prompt
    context (str): Additional context to resolve ambiguity

    Returns:
    str: The AI's response to the clarified prompt
    """
    clarified_prompt = f"{context}\n\nBased on this context, {prompt}"
    return llm.invoke(clarified_prompt).content

In [ ]:
# Example usage
ambiguous_prompt = "Tell me about the bank."
contexts = [
    "You are a financial advisor discussing savings accounts.",
    "You are a geographer describing river formations."
]

for context in contexts:
    print(f"Context: {context}")
    print(f"Clarified response: {resolve_ambiguity(ambiguous_prompt, context)}")
    print("-" * 50)

Context: You are a financial advisor discussing savings accounts.
Clarified response: I don’t have details on a specific bank from your message, but here’s a practical overview of what a typical bank savings account offers and how to evaluate it as a financial advisor would.

What a savings account usually provides
- Safety: Funds are insured (in the U.S., up to $250,000 per depositor per bank by the FDIC; credit unions have NCUA insurance).
- Liquidity: Easy access to your money for emergencies, transfers, or deposits.
- Interest: Earns interest (APY), though rates vary widely by bank and account type.
- Predictable risk: Very low risk compared with investments like stocks or bonds.

Key terms to compare when evaluating a bank’s savings account
- APY (annual percentage yield) and compounding: How much you earn and how often interest is added (daily, monthly, etc.).
- Minimums: Opening deposit required and the minimum balance needed to earn the advertised rate or avoid fees.
- Fees: Mo

One way to deal with abiguous user prompts is to use additional information as context of the prompt like th eprevious examples.

## Techniques for Writing Clearer Prompts

In [ ]:
def compare_prompt_clarity(original_prompt, improved_prompt):
    """
    Compare the responses to an original prompt and an improved, clearer version.

    Args:
    original_prompt (str): The original, potentially unclear prompt
    improved_prompt (str): An improved, clearer version of the prompt

    Returns:
    tuple: Responses to the original and improved prompts
    """
    original_response = llm.invoke(original_prompt).content
    improved_response = llm.invoke(improved_prompt).content
    return original_response, improved_response

In [ ]:
# Example usage
original_prompt = "How do I make it?"
improved_prompt = "Provide a step-by-step guide for making a classic margherita pizza, including ingredients and cooking instructions."

original_response, improved_response = compare_prompt_clarity(original_prompt, improved_prompt)

print("Original Prompt Response:")
print(original_response)
print("\nImproved Prompt Response:")
print(improved_response)

Original Prompt Response:
I’d be happy to help, but I’m missing what “it” is. Could you tell me what you want to make? A few quick questions to zero in:

- What is the item or project called (e.g., a recipe, a DIY item, a software feature, a craft, etc.)?
- Do you have any time limits or budget, and what skill level should I tailor it to?
- What tools or ingredients do you already have?
- Any dietary, safety, or other constraints?
- Do you want step-by-step instructions or a high-level plan?

If you’d like, you can also pick from these quick-start options and I’ll tailor it:

- Cooking/Baking: a simple recipe with ingredients you have.
- Craft/DIY: a basic project (e.g., a shelf, a lamp, paper crafts).
- Tech/Software: a small feature or app you want to build.
- General how-to: a plan you can follow for almost any project.

Here’s a handy template you can fill in now:
- Goal: What are we making, and what would a successful result look like?
- Materials/tools: List what you have or need

## Structured Prompts for Clarity

Using structured prompts can significantly improve clarity and consistency in AI responses.

In [ ]:
structured_prompt = PromptTemplate(
    input_variables=["topic", "aspects", "tone"],
    template="""Provide an analysis of {topic} considering the following aspects:
    1. {{aspects[0]}}
    2. {{aspects[1]}}
    3. {{aspects[2]}}

    Present the analysis in a {tone} tone.
    """
)

In [ ]:
# Example usage
input_variables = {
    "topic": "the impact of social media on society",
    "aspects": ["communication patterns", "mental health", "information spread"],
    "tone": "balanced and objective"
}

chain = structured_prompt | llm
response = chain.invoke(input_variables).content
print(response)

It looks like the three aspects you wanted to examine were placeholders. I’ll proceed with a balanced analysis using three widely discussed dimensions of social media’s impact. If you’d prefer different aspects, tell me and I’ll tailor accordingly.

1) Mental health and well-being
- Positive contributions:
  - Facilitates social connection, especially for people with limited offline access to support networks.
  - Provides peer support, information about health resources, and community for marginalized or niche groups.
  - Real-time communication can aid crisis response and relief efforts.
- Negative contributions:
  - Social comparison, envy, and reduced self-esteem from curated highlight reels.
  - Exposure to harassment, cyberbullying, and online abuse, particularly for younger users and minority groups.
  - Sleep disruption, attention fragmentation, and potential compulsive or addictive usage patterns.
- Moderating factors:
  - Platform design (notifications, algorithms, recommenda

## Improving Prompt Clarity Automatically

In [ ]:
def improve_prompt_clarity(unclear_prompt):
    """
    Improve the clarity of a given prompt.

    Args:
    unclear_prompt (str): The original unclear prompt

    Returns:
    str: An improved, clearer version of the prompt
    """
    improvement_prompt = f"The following prompt is unclear: '{unclear_prompt}'. Please provide a clearer, more specific version of this prompt. output just the improved prompt and nothing else."
    return llm.invoke(improvement_prompt).content

In [ ]:
unclear_prompts = [
    "What's the difference?",
    "How does it work?",
    "Why is it important?"
]

for prompt in unclear_prompts:
    improved_prompt = improve_prompt_clarity(prompt)
    print(f"Original: {prompt}")
    print(f"Improved: {improved_prompt}")
    print("-" * 50)

Original: What's the difference?
Improved: Please compare [Item 1] and [Item 2], focusing on their differences. For each item, cover: (a) intended purpose and audience, (b) core features and capabilities, (c) advantages and disadvantages, (d) typical use cases and examples, (e) cost, pricing model, and availability/licensing, (f) performance, reliability, and scalability (if relevant), (g) limitations, caveats, and common pitfalls. Present the differences in a clear, structured format (use bullet points under each category). Conclude with a concise recommendation about which item is preferable in specific scenarios, and note any assumptions or uncertainties. If there are more than two items, adapt the structure accordingly. If essential details are missing, ask for clarification.
--------------------------------------------------
Original: How does it work?
Improved: Explain in clear, concrete terms how [the subject] works. Provide: 1) a brief high-level purpose and outcome; 2) a map o

## Conclusions

### Trade-offs

#### Reducing ambiguity
- Increases consistency and reproducibility of model outputs.
- Improves reliability and makes behavior easier to debug.
- Can reduce creativity if prompts become overly restrictive.

#### Allowing ambiguity
- Enables creative, exploratory, and diverse outputs.
- Encourages ideation and alternative perspectives.
- Increases variance and makes evaluation more difficult.

### When to apply it

#### Favor clarity and low ambiguity
- Tasks with a clear objective or single ground truth.
- Outputs that are consumed by automated systems.
- Extraction, classification, and decision-oriented use cases.

#### Allow controlled ambiguity
- Creative or exploratory tasks.
- Brainstorming and ideation scenarios.
- Situations where human judgment is applied downstream.

### When NOT to apply it

#### Avoid ambiguity
- When strict formats or schemas are required.
- When deterministic behavior is necessary.
- When errors can propagate into downstream systems.

#### Avoid excessive clarity
- When creativity and emergent reasoning are desired.
- When multiple valid interpretations exist.
- When over-specification may bias the model.

### Clarifying terms

#### Ambiguity
- Prompts that allow multiple valid interpretations.
- A deliberate design choice based on task goals.

#### Clarity
- Prompts that narrow interpretation through explicit intent and constraints.
- Improves alignment but does not guarantee correctness.

#### Key distinction
- The objective is not to eliminate ambiguity everywhere.
- Clarity and ambiguity should be applied intentionally depending on the use case.
